In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go


##### Merge 12 months of sales data into a single csv file

In [3]:
files =[file for file in os.listdir("/content/drive/MyDrive/sales_data_analyses/")]
for file in files:
    print(file)

Sales_June_2019.csv
Sales_November_2019.csv
Sales_March_2019.csv
Sales_May_2019.csv
Sales_February_2019.csv
Sales_April_2019.csv
Sales_January_2019.csv
Sales_July_2019.csv
Sales_August_2019.csv
Sales_October_2019.csv
Sales_September_2019.csv
Sales_December_2019.csv
all_data.csv


In [4]:
path = "/content/drive/MyDrive/sales_data_analyses"

#blank dataframe
all_data = pd.DataFrame()

for file in files:
    current_df = pd.read_csv(path+"/"+file)
    all_data = pd.concat([all_data, current_df])

all_data.shape

(560550, 6)

##### convert it into dataset

In [5]:
all_data.to_csv('/content/drive/MyDrive/sales_data_analyses/all_data.csv',index=False)

##### Data cleaning and formatting

In [6]:
all_data.dtypes

Order ID            object
Product             object
Quantity Ordered    object
Price Each          object
Order Date          object
Purchase Address    object
dtype: object

In [7]:
all_data.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,209921,USB-C Charging Cable,1,11.95,06/23/19 19:34,"950 Walnut St, Portland, ME 04101"
1,209922,Macbook Pro Laptop,1,1700.0,06/30/19 10:05,"80 4th St, San Francisco, CA 94016"
2,209923,ThinkPad Laptop,1,999.99,06/24/19 20:18,"402 Jackson St, Los Angeles, CA 90001"
3,209924,27in FHD Monitor,1,149.99,06/05/19 10:21,"560 10th St, Seattle, WA 98101"
4,209925,Bose SoundSport Headphones,1,99.99,06/25/19 18:58,"545 2nd St, San Francisco, CA 94016"


In [8]:
all_data.isnull().sum()

Order ID            1635
Product             1635
Quantity Ordered    1635
Price Each          1635
Order Date          1635
Purchase Address    1635
dtype: int64

In [9]:
all_data = all_data.dropna(how='all')
all_data.shape

(558915, 6)

##### What is the best month for sale?

In [10]:
'04/19/19 08:46'.split('/')[0]

'04'

In [11]:
def month(x):
    return x.split('/')[0]

##### add month col

In [12]:
all_data['Month']=all_data['Order Date'].apply(month)

In [13]:
all_data.dtypes

Order ID            object
Product             object
Quantity Ordered    object
Price Each          object
Order Date          object
Purchase Address    object
Month               object
dtype: object

In [14]:
all_data['Month']=all_data['Month'].astype(int)

ValueError: ignored

In [15]:
print(all_data['Month'].unique())


['06' 'Order Date' '07' '11' '12' '03' '04' '05' '02' '01' '08' '09' '10']


In [16]:
all_data['Month'].unique()

array(['06', 'Order Date', '07', '11', '12', '03', '04', '05', '02', '01',
       '08', '09', '10'], dtype=object)

In [17]:
filter=all_data['Month']=='Order Date'
len(all_data[~filter])

557850

In [18]:
all_data=all_data[~filter]

In [19]:
all_data.shape

(557850, 7)

In [20]:
all_data.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Month
0,209921,USB-C Charging Cable,1,11.95,06/23/19 19:34,"950 Walnut St, Portland, ME 04101",06
1,209922,Macbook Pro Laptop,1,1700.0,06/30/19 10:05,"80 4th St, San Francisco, CA 94016",06
2,209923,ThinkPad Laptop,1,999.99,06/24/19 20:18,"402 Jackson St, Los Angeles, CA 90001",06
3,209924,27in FHD Monitor,1,149.99,06/05/19 10:21,"560 10th St, Seattle, WA 98101",06
4,209925,Bose SoundSport Headphones,1,99.99,06/25/19 18:58,"545 2nd St, San Francisco, CA 94016",06


In [21]:
all_data['Month']=all_data['Month'].astype(int)

In [22]:
all_data.dtypes

Order ID            object
Product             object
Quantity Ordered    object
Price Each          object
Order Date          object
Purchase Address    object
Month                int64
dtype: object

In [23]:
all_data['Price Each']=all_data['Price Each'].astype(float)

In [24]:
all_data['Quantity Ordered']=all_data['Quantity Ordered'].astype(int)

In [25]:
all_data['sales']=all_data['Quantity Ordered']*all_data['Price Each']
all_data.head(5)

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Month,sales
0,209921,USB-C Charging Cable,1,11.95,06/23/19 19:34,"950 Walnut St, Portland, ME 04101",6,11.95
1,209922,Macbook Pro Laptop,1,1700.00,06/30/19 10:05,"80 4th St, San Francisco, CA 94016",6,1700.00
2,209923,ThinkPad Laptop,1,999.99,06/24/19 20:18,"402 Jackson St, Los Angeles, CA 90001",6,999.99
3,209924,27in FHD Monitor,1,149.99,06/05/19 10:21,"560 10th St, Seattle, WA 98101",6,149.99
4,209925,Bose SoundSport Headphones,1,99.99,06/25/19 18:58,"545 2nd St, San Francisco, CA 94016",6,99.99


In [26]:
all_data.groupby('Month')['sales'].sum()

Month
1      5466770.19
2      6606067.26
3      8421301.14
4     10172010.72
5      9457820.25
6      7733406.78
7      7943327.28
8      6733403.64
9      6292680.39
10    11210180.64
11     9598809.60
12    13840330.02
Name: sales, dtype: float64

In [27]:
fig = px.bar(all_data.groupby('Month')['sales'].sum().reset_index(), x='Month', y='sales',
             labels={'Month': 'Month', 'sales': 'Sales in USD ($)'},
             title='Monthly Sales Data')

fig.show()

##### Which city has max order

In [28]:
'917 1st St, Dallas, TX 75001'.split(',')[1]

' Dallas'

In [29]:
def city(x):
    return x.split(',')[1]

In [30]:
all_data['city']=all_data['Purchase Address'].apply(city)

In [31]:
all_data.groupby('city')['city'].count()

city
 Atlanta           44643
 Austin            29715
 Boston            59802
 Dallas            44460
 Los Angeles       88815
 New York City     74628
 Portland          37395
 San Francisco    134196
 Seattle           44196
Name: city, dtype: int64

In [32]:
city_count = all_data.groupby('city')['city'].count()

fig = go.Figure([go.Bar(x=city_count.index, y=city_count)])

fig.update_layout(
    title="Number of orders received by city",
    xaxis_title="City names",
    yaxis_title="Received orders"
)

fig.show()


##### What time should we display advertisements to maximise for product purchase?

In [33]:
all_data['Order Date'][0].dtype

dtype('O')

In [34]:
all_data['Hour'] = pd.to_datetime(all_data['Order Date']).dt.hour

In [35]:
keys=[]
hour=[]
for key,hour_df in all_data.groupby('Hour'):
    keys.append(key)
    hour.append(len(hour_df))

In [36]:
fig = px.line(x=keys, y=hour, labels={'x': 'Hour of the day', 'y': 'Number of orders'}, title='Orders per Hour')
fig.update_xaxes(nticks=24)
fig.show()

##### between 12pm and 7pm is probably the best time to advertise to maximise product purchase

#### What product sold the most? &  Why?

In [37]:
product_qty = all_data.groupby('Product')['Quantity Ordered'].sum().reset_index()

fig = px.bar(product_qty, x='Product', y='Quantity Ordered', color='Product', height=500)
fig.update_layout(title='Total Quantity Ordered by Product', xaxis_title='Product', yaxis_title='Quantity Ordered')
fig.show()


In [38]:
all_data.groupby('Product')['Price Each'].mean()

Product
20in Monitor                   109.99
27in 4K Gaming Monitor         389.99
27in FHD Monitor               149.99
34in Ultrawide Monitor         379.99
AA Batteries (4-pack)            3.84
AAA Batteries (4-pack)           2.99
Apple Airpods Headphones       150.00
Bose SoundSport Headphones      99.99
Flatscreen TV                  300.00
Google Phone                   600.00
LG Dryer                       600.00
LG Washing Machine             600.00
Lightning Charging Cable        14.95
Macbook Pro Laptop            1700.00
ThinkPad Laptop                999.99
USB-C Charging Cable            11.95
Vareebadd Phone                400.00
Wired Headphones                11.99
iPhone                         700.00
Name: Price Each, dtype: float64

In [39]:
products=all_data.groupby('Product')['Quantity Ordered'].sum().index
quantity=all_data.groupby('Product')['Quantity Ordered'].sum()
prices=all_data.groupby('Product')['Price Each'].mean()

In [40]:
fig = go.Figure()

fig.add_trace(
    go.Bar(x=products, y=quantity, name='Quantity', marker_color='green')
)

fig.add_trace(
    go.Scatter(x=products, y=prices, name='Price', yaxis='y2', line=dict(color='blue'))
)

fig.update_layout(
    title='Quantity and Price of Products',
    xaxis=dict(title='Product'),
    yaxis=dict(title='Quantity'),
    yaxis2=dict(title='Price', overlaying='y', side='right'),
    height=600, width=1000
)

fig.show()


##### The top selling product is 'AAA Batteries'. The top selling products seem to have a correlation with the price of the product. The cheaper the product higher the quantity ordered and vice versa.

In [41]:
all_data.shape

(557850, 10)

##### What products are most often sold together?

#### note: keep orders that have same order Id,are sold mostly together

In [42]:
df=all_data[all_data['Order ID'].duplicated(keep=False)]
df.head(20)

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Month,sales,city,Hour
0,209921,USB-C Charging Cable,1,11.95,06/23/19 19:34,"950 Walnut St, Portland, ME 04101",6,11.95,Portland,19
1,209922,Macbook Pro Laptop,1,1700.00,06/30/19 10:05,"80 4th St, San Francisco, CA 94016",6,1700.00,San Francisco,10
2,209923,ThinkPad Laptop,1,999.99,06/24/19 20:18,"402 Jackson St, Los Angeles, CA 90001",6,999.99,Los Angeles,20
3,209924,27in FHD Monitor,1,149.99,06/05/19 10:21,"560 10th St, Seattle, WA 98101",6,149.99,Seattle,10
4,209925,Bose SoundSport Headphones,1,99.99,06/25/19 18:58,"545 2nd St, San Francisco, CA 94016",6,99.99,San Francisco,18
5,209926,Apple Airpods Headphones,1,150.00,06/28/19 20:04,"386 Lake St, Seattle, WA 98101",6,150.00,Seattle,20
6,209927,Lightning Charging Cable,1,14.95,06/28/19 00:07,"29 Lincoln St, Los Angeles, CA 90001",6,14.95,Los Angeles,0
7,209928,Apple Airpods Headphones,1,150.00,06/16/19 21:30,"350 9th St, New York City, NY 10001",6,150.00,New York City,21
8,209929,Wired Headphones,1,11.99,06/28/19 10:56,"612 Meadow St, Portland, OR 97035",6,11.99,Portland,10
9,209930,27in FHD Monitor,1,149.99,06/02/19 11:22,"625 Ridge St, Los Angeles, CA 90001",6,149.99,Los Angeles,11


In [43]:
#create grouped col
df['Grouped'] = df.groupby('Order ID')['Product'].transform(lambda x: ','.join(x))

In [44]:
df.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Month,sales,city,Hour,Grouped
0,209921,USB-C Charging Cable,1,11.95,06/23/19 19:34,"950 Walnut St, Portland, ME 04101",6,11.95,Portland,19,"USB-C Charging Cable,USB-C Charging Cable,USB-..."
1,209922,Macbook Pro Laptop,1,1700.00,06/30/19 10:05,"80 4th St, San Francisco, CA 94016",6,1700.00,San Francisco,10,"Macbook Pro Laptop,Macbook Pro Laptop,Macbook ..."
2,209923,ThinkPad Laptop,1,999.99,06/24/19 20:18,"402 Jackson St, Los Angeles, CA 90001",6,999.99,Los Angeles,20,"ThinkPad Laptop,ThinkPad Laptop,ThinkPad Laptop"
3,209924,27in FHD Monitor,1,149.99,06/05/19 10:21,"560 10th St, Seattle, WA 98101",6,149.99,Seattle,10,"27in FHD Monitor,27in FHD Monitor,27in FHD Mon..."
4,209925,Bose SoundSport Headphones,1,99.99,06/25/19 18:58,"545 2nd St, San Francisco, CA 94016",6,99.99,San Francisco,18,"Bose SoundSport Headphones,Bose SoundSport Hea..."


In [45]:
df.shape

(557850, 11)

In [46]:
#lets drop out all duplicate Order ID
df2 = df.drop_duplicates(subset=['Order ID'])

In [47]:
# get the value counts of the "Grouped" column and select the top 5
top_values = df2['Grouped'].value_counts().nlargest(5)

# create the pie chart using Plotly
fig = go.Figure(data=[go.Pie(labels=top_values.index, values=top_values.values)])

fig.show()


In [48]:
import plotly.graph_objs as go
from plotly.offline import iplot

In [49]:
values=df2['Grouped'].value_counts()[0:5]
labels=df['Grouped'].value_counts()[0:5].index

In [50]:
trace=go.Pie(labels=labels, values=values,
               hoverinfo='label+percent', textinfo='value',
               textfont=dict(size=25),
              pull=[0, 0, 0,0.2, 0]
               )

In [51]:
iplot([trace])